In [ ]:
from config import Config
from models import create_llm 
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate,PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from pydantic import BaseModel, Field

llm = create_llm(Config.MODEL) 

In [21]:
class OverpassQueryModel(BaseModel):
    area_name: str = Field(..., description="The name of the area to be searched, e.g., 'France', 'Paris', etc.")
    admin_level: int = Field(..., description="The OSM administrative level to search for, e.g., 2 for countries, 4 for regions.")
    overpass_query: str = Field(..., description="A complete Overpass QL query for retrieving the requested administrative boundaries.")




In [ ]:
prompt_template = """

You are a geospatial data engineer specializing in OpenStreetMap (OSM) and the Overpass API.
Your task is to generate a valid Overpass QL query that returns full polygon boundaries as relation elements for a given area name and admin level.

The query must:

    Look up the area using area[name=...] (and optionally fall back to name:en, name:ar, etc.)

    Filter only relation elements with the specified admin_level and boundary=administrative

    Output complete polygon geometry (not centroids or points)

    Use proper Overpass syntax like out body; >; out skel qt; or out geom

    Be formatted correctly for Overpass Turbo or any standard Overpass client

Return only the complete working query as your output.

{format_instructions}

Only output the raw JSON object. Nothing else.

User query: {query}

"""

parser = PydanticOutputParser(pydantic_object=OverpassQueryModel)

prompt= SystemMessagePromptTemplate.from_template
# prompt = PromptTemplate(

#     template=prompt_template,

#     input_variables=["query"],

#     partial_variables={"format_instructions": parser.get_format_instructions()}

# )



chain = LLMChain(llm= llm, prompt=prompt)

query = "show me departments of france"

result = chain.invoke({"query": query}) 

result

{'query': 'show me departments of france',
 'text': '{"properties": {"area_name": "France", "admin_level": 2, "overpass_query": "relation[type=\\"boundary\\"](area[name=france] or name:en, area[name=ar,france] or name:es,france) in France[admin_level=2][boundary=administrative]; out body; >; out skel qt;"}}'}

In [20]:
structured_output = parser.parse(result['text'])
print(structured_output.overpass_query)


OutputParserException: Failed to parse OverpassQueryModel from completion {"properties": {"area_name": "France", "admin_level": 2, "overpass_query": "relation[type=\"boundary\"](area[name=france] or name:en, area[name=ar,france] or name:es,france) in France[admin_level=2][boundary=administrative]; out body; >; out skel qt;"}}. Got: 3 validation errors for OverpassQueryModel
area_name
  Field required [type=missing, input_value={'properties': {'area_nam...body; >; out skel qt;'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
admin_level
  Field required [type=missing, input_value={'properties': {'area_nam...body; >; out skel qt;'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
overpass_query
  Field required [type=missing, input_value={'properties': {'area_nam...body; >; out skel qt;'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubl

In [17]:
import requests

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# Example Overpass QL query to fetch admin_level=2 boundaries with ISO country codes
query = structured_output.overpass_query

def run_overpass_query(query: str):
    response = requests.post(OVERPASS_URL, data={"data": query})
    
    if response.status_code == 200:
        return response.json()  # parsed JSON
    else:
        raise Exception(f"Overpass query failed with status {response.status_code}:\n{response.text}")
try:
    result = run_overpass_query(query)
    print(f"Returned {len(result.get('elements', []))} elements.")
    # Optional: Save to file
    # import json
    # with open("overpass_output.json", "w") as f:
    #     json.dump(result, f, indent=2)
except Exception as e:
    print("Error:", e)    


NameError: name 'structured_output' is not defined